In [2]:
import enum
from collections import Counter, defaultdict
from ntpath import join
import numpy as np
import pandas as pd
from sqlalchemy import column
import xlsxwriter
import os
import seaborn as sns
from nltk import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
nltk.download('omw-1.4')
from pathlib import Path
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import LabelEncoder
from matplotlib.gridspec import GridSpec
import contractions 
from pathlib import Path
from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.io import imread, imshow
from skimage import io
import json
from skimage import img_as_float
from skimage.transform import rescale, resize
from itertools import product
from PIL import Image
from clean_tabular import CleanData, CleanImages



[nltk_data] Downloading package omw-1.4 to /Users/aj8/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [57]:
pd.set_option('max_colwidth', 200)
pd.set_option('display.max_columns', 20)
class ProductDescpMannual(CleanData):
    def __init__(self):
        super().__init__(tab_names=['Products'])
        self.label_len = len(self.major_map_decoder)
        self.product_frame = self.table_dict['Products'].copy()

    def clean_prod(self, col='product_description'):
        stop_words = set(stopwords.words('english'))
        word_lemmitizer = WordNetLemmatizer()
        punct_re = re.compile(r'[^A-Za-z \n-]') #[^A-Za-z0-9 .]
        self.product_frame[col] = self.product_frame[col].copy().str.replace(punct_re, '')
        all_text_ls = []
        main_ls = []
        for i in self.product_frame[col]:
            dum_ls = i.split()
            dum_ls = [i for i in dum_ls if i not in  stop_words]
            dum_ls = [i.replace('\n', ' ') for i in dum_ls]
            dum_ls = [i.strip('\-') for i in dum_ls]
            dum_ls = [word_lemmitizer.lemmatize(i) for i in dum_ls]
            dum_ls = [contractions.fix(j) for j in dum_ls]
            dum_ls = [i for i in dum_ls if i.isalpha and len(i)>2]
            main_ls.append(' '.join(dum_ls))
            # print(' '.join(dum_ls))
            all_text_ls.extend(dum_ls)
        all_text_ls = [i.lower() for i in all_text_ls]
        self.product_frame[col] = main_ls 
        self.product_frame[col] = self.product_frame[col].apply(lambda i: i.lower())
        print(len(all_text_ls))
        return all_text_ls, len(all_text_ls)
    
    def word_freq(self, col='product_description', num_items=100):
        all_words = self.clean_prod(col=col)[0]
        count_ls = Counter(all_words)
        count_dict = {i[0]: i[1] for i in count_ls.most_common(num_items)}
        return count_dict

    def get_word_set(self, col='product_description'):
        if col=='product_name':
            self.clean_prod_name(col=col)
        self.clean_prod(col=col)
        full_word_ls = []
        for i in self.product_frame[col]:
            full_word_ls.extend(i.split())
        full_word_set = list(set(full_word_ls))
        return full_word_ls, full_word_set ,len(full_word_set)+1


    def vocab_encoder(self, col='product_description', limit_it=False, vocab_limit=30000):
        word_ls, word_set, full_vocab_size = self.get_word_set(col)
        if limit_it==False:
            vocab_size = full_vocab_size
            word_encoder = defaultdict(lambda: vocab_size-1)
            current_word_encoder = {key[0]: integer for integer, key in enumerate(Counter(word_ls).most_common()[:vocab_size-1])}
            word_encoder['<UNKNOWN>'] = vocab_size-1
            word_encoder.update(current_word_encoder)
            word_decoder = {val: key for key, val in word_encoder.items()}
        else:
            vocab_size=vocab_limit
            word_encoder = defaultdict(lambda: vocab_size)
            current_word_encoder = {key[0]: integer for integer, key in enumerate(Counter(word_ls).most_common()[:vocab_size])}
            word_encoder['<UNKNOWN>'] = vocab_size
            word_encoder.update(current_word_encoder)
            word_decoder = {val: key for key, val in word_encoder.items()}
        
        # word_encoder = {val: key for key, val in word_decoder.items()}
        return word_encoder, word_decoder, self.product_frame, word_ls

    
    def dataloader_preprocess(self, limit_it=True, vocab_lim=10000, context_size=5):
        product_encoder, product_decoder, DF, total_wrd_ls = self.vocab_encoder(limit_it=limit_it, vocab_limit=vocab_lim)
        df = DF.copy()
        init_ls = []
        for i in range(len(df)):
            prod_descript = df['product_description'].iloc[i].split()
            init_ls.append([
                [
                    list(np.array([[prod_descript[i - j], prod_descript[i+j]] for j in range(1, context_size+1)]).flatten()),
                    prod_descript[i]
                ]
                for i in range(context_size, len(prod_descript)-context_size)
            ])
        df['original_context_target'] = init_ls
        df = df.explode('original_context_target').reset_index()
        for idx, val in enumerate(df['original_context_target']):
            if type(val) is not list:
                df.drop(idx, axis=0, inplace=True)
        
        def stub_coder(ls, code_type='encoded'):
            code_dict = {'encoded': product_encoder, 'decoded': product_decoder}
            new_ls = []
            for i in ls.copy():
                new_ls.append(code_dict[code_type][i])
            return new_ls

        df = pd.concat([df, pd.DataFrame(df['original_context_target'].tolist())], axis=1)
        df.rename(columns={0: 'context', 1: 'target'}, inplace=True)
        df.dropna(inplace=True)
        df['context_encoded'] = df['context'].apply(lambda i: stub_coder(ls=i, code_type='encoded'))
        df['context_decoded'] = df['context_encoded'].apply(lambda i: stub_coder(ls=i,code_type='decoded'))
        df['target_encoded'] = df['target'].apply(lambda i: product_encoder[i])
        df['target_decoded'] = df['target_encoded'].apply(lambda i: product_decoder[i])
        print(df)
        print(df.columns)
        writ = pd.ExcelWriter(Path(Path.cwd(), 'data_files', 'new_sampler.xlsx'), engine='xlsxwriter')
        with writ as writer:
            df.iloc[:9000].to_excel(writ)
        self.product_frame = df.copy()
        return self.product_frame, product_encoder, product_decoder, total_wrd_ls





In [58]:
prod = ProductDescpMannual()
prod.dataloader_preprocess()

Encoder {'Home & Garden ': 0, 'Baby & Kids Stuff ': 1, 'DIY Tools & Materials ': 2, 'Music, Films, Books & Games ': 3, 'Phones, Mobile Phones & Telecoms ': 4, 'Clothes, Footwear & Accessories ': 5, 'Other Goods ': 6, 'Health & Beauty ': 7, 'Sports, Leisure & Travel ': 8, 'Appliances ': 9, 'Computers & Software ': 10, 'Office Furniture & Equipment ': 11, 'Video Games & Consoles ': 12}
215807
         index                                    id  \
0          1.0  243809c0-9cfc-4486-ad12-3b7a16605ba9   
1          1.0  243809c0-9cfc-4486-ad12-3b7a16605ba9   
2          1.0  243809c0-9cfc-4486-ad12-3b7a16605ba9   
3          1.0  243809c0-9cfc-4486-ad12-3b7a16605ba9   
4          1.0  243809c0-9cfc-4486-ad12-3b7a16605ba9   
...        ...                                   ...   
148314  7974.0  5d53b1b6-5dc7-4f18-9c09-4cd686051c5a   
148315  7974.0  5d53b1b6-5dc7-4f18-9c09-4cd686051c5a   
148316  7974.0  5d53b1b6-5dc7-4f18-9c09-4cd686051c5a   
148317  7974.0  5d53b1b6-5dc7-4f18-9c09-4cd686

In [ ]:
def generate_batches(batch_size=32, context_size = 4, threshold=1e-5):
    product_description_class = ProductDescpMannual()
    DF, product_encoder, product_decoder, total_wrd_ls = product_description_class.dataloader_preprocess(limit_it=True, vocab_limit=10000, context_size=4)
    df = DF.copy()
    num_batches = len(df)//batch_size
    corpus_length = len(total_wrd_ls)
    df = df.iloc[:num_batches*batch_size]
    wrd_counter = Counter(total_wrd_ls)
    wrd_freq_dict = {wrd: count/len(total_wrd_ls) for wrd, count in wrd_counter.items()}
    wrd_drop_proba = {wrd: 1- np.sqrt(threshold/wrd_freq_dict[wrd]) for wrd in wrd_counter.keys()}
    
    for idx in range(0, len(df), batch_size):
        pa
    

In [ ]:
def train_model():
    product_description_class = ProductDescpMannual()
    DF, product_encoder, product_decoder, total_wrd_ls = product_description_class.dataloader_preprocess(limit_it=True, vocab_limit=10000, context_size=4)
    df = DF.copy()
    corpus_length = len(total_wrd_ls)
